In [30]:
## preprocessing.py imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import set_config
import joblib
import xgboost
import multipart

#custom 
from smackbang.preprocess import DateFormatter, DateEncoder, TimeFeaturesEncoder, haversine_vectorized, DistanceTransformer, duration_process, set_preproc_pipeline
from smackbang.predict import process_matches

set_config(display='diagram')



In [37]:
import requests
import pandas as pd
from requests.structures import CaseInsensitiveDict

url = "https://smackbang-image-w76hg6ifha-ew.a.run.app/matches"

headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

origin_one = 'NRT'
origin_two = 'SYD'
departure_date = '01/04/2022'
continent='AS'

query_string = {'origin_one':origin_one, 'origin_two':origin_two, 'departure_date':departure_date, 
                'continent':continent, 'return_date':'', 'currency':'USD'}

result = requests.get(url, headers=headers, params=query_string).json()
                
matches_df = pd.DataFrame(result)

In [38]:
matches_df

,price_Sydney,price_Tokyo,duration_Sydney,duration_Tokyo,total_stops_Sydney,total_stops_Tokyo,distance_Sydney,distance_Tokyo,local_departure_Sydney,local_departure_Tokyo,local_arrival_Sydney,local_arrival_Tokyo,deep_link_Sydney,deep_link_Tokyo,combined_price_,combined_duration_,lat_,lon_
Hanoi,365,322,18.00,6.33,1,0,7798.15,3723.95,2022-04-01T13:25:00.000Z,2022-04-01T08:55:00.000Z,2022-04-02T11:25:00.000Z,2022-04-01T13:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=HAN&flig...,https://www.kiwi.com/deep?from=NRT&to=HAN&flig...,687,24.33,21.221201,105.806999
Manila,428,273,8.33,4.75,0,0,6269.74,3054.20,2022-04-01T11:15:00.000Z,2022-04-01T13:10:00.000Z,2022-04-01T16:50:00.000Z,2022-04-01T17:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=MNL&flig...,https://www.kiwi.com/deep?from=NRT&to=MNL&flig...,701,13.08,14.508600,121.019997
Bangkok,349,385,9.83,6.50,0,0,7523.15,4653.65,2022-04-01T10:50:00.000Z,2022-04-01T12:00:00.000Z,2022-04-01T16:40:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=BKK&flig...,https://www.kiwi.com/deep?from=NRT&to=BKK&flig...,734,16.33,13.681100,100.747002
Ho Chi Minh City,501,237,8.92,7.00,0,0,6853.68,4388.16,2022-04-01T11:15:00.000Z,2022-04-01T08:55:00.000Z,2022-04-01T16:10:00.000Z,2022-04-01T13:55:00.000Z,https://www.kiwi.com/deep?from=SYD&to=SGN&flig...,https://www.kiwi.com/deep?from=NRT&to=SGN&flig...,738,15.92,10.818800,106.652000
Taipei,373,383,19.33,3.67,1,0,7293.46,2183.46,2022-04-01T13:25:00.000Z,2022-04-01T13:50:00.000Z,2022-04-02T05:45:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=TPE&flig...,https://www.kiwi.com/deep?from=NRT&to=TPE&flig...,756,23.00,25.077700,121.233002
Osaka,741,85,12.33,1.50,1,0,7824.93,462.63,2022-04-01T21:45:00.000Z,2022-04-01T07:00:00.000Z,2022-04-02T08:05:00.000Z,2022-04-01T08:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=ITM&flig...,https://www.kiwi.com/deep?from=NRT&to=ITM&flig...,826,13.83,34.785500,135.438004
Kuala Lumpur,318,536,8.83,14.67,0,1,6591.75,5414.54,2022-04-01T06:25:00.000Z,2022-04-01T19:00:00.000Z,2022-04-01T21:35:00.000Z,2022-04-02T08:40:00.000Z,https://www.kiwi.com/deep?from=SYD&to=KUL&flig...,https://www.kiwi.com/deep?from=NRT&to=KUL&flig...,854,23.50,2.745580,101.709999
Fukuoka,750,111,13.08,2.08,1,0,7823.25,940.88,2022-04-01T21:45:00.000Z,2022-04-01T07:10:00.000Z,2022-04-02T08:50:00.000Z,2022-04-01T09:15:00.000Z,https://www.kiwi.com/deep?from=SYD&to=FUK&flig...,https://www.kiwi.com/deep?from=NRT&to=FUK&flig...,861,15.16,33.585899,130.451004
Chiang Mai,539,429,26.92,10.58,1,1,8064.84,4471.46,2022-04-01T16:00:00.000Z,2022-04-01T12:00:00.000Z,2022-04-02T14:55:00.000Z,2022-04-01T20:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=CNX&flig...,https://www.kiwi.com/deep?from=NRT&to=CNX&flig...,968,37.50,18.766800,98.962601
Da Nang,550,428,27.75,9.83,1,1,7190.96,3869.90,2022-04-01T16:00:00.000Z,2022-04-01T09:15:00.000Z,2022-04-02T15:45:00.000Z,2022-04-01T17:05:00.000Z,https://www.kiwi.com/deep?from=SYD&to=DAD&flig...,https://www.kiwi.com/deep?from=NRT&to=DAD&flig...,978,37.58,16.043900,108.198998


In [39]:
city_one = 'Sydney'
city_two = 'Tokyo'

In [40]:
df1_formatted, df2_formatted = process_matches(matches_df,city_one,city_two)

In [43]:
df1_formatted

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,origin_one_latitude,origin_one_longitude,origin_two_latitude,origin_two_longitude
0,Multiple Carriers,01/04/2022,Sydney,Hanoi,Sydney-Hanoi,13:25,11:25,18h 0m,1,No info,21.221201,105.806999,-33.946098,151.177002
1,Multiple Carriers,01/04/2022,Sydney,Manila,Sydney-Manila,11:15,16:50,8h 20m,0,No info,14.508600,121.019997,-33.946098,151.177002
2,Multiple Carriers,01/04/2022,Sydney,Bangkok,Sydney-Bangkok,10:50,16:40,9h 50m,0,No info,13.681100,100.747002,-33.946098,151.177002
3,Multiple Carriers,01/04/2022,Sydney,Ho Chi Minh City,Sydney-Ho Chi Minh City,11:15,16:10,8h 55m,0,No info,10.818800,106.652000,-33.946098,151.177002
4,Multiple Carriers,01/04/2022,Sydney,Taipei,Sydney-Taipei,13:25,05:45,19h 20m,1,No info,25.077700,121.233002,-33.946098,151.177002
5,Multiple Carriers,01/04/2022,Sydney,Osaka,Sydney-Osaka,21:45,08:05,12h 20m,1,No info,34.785500,135.438004,-33.946098,151.177002
6,Multiple Carriers,01/04/2022,Sydney,Kuala Lumpur,Sydney-Kuala Lumpur,06:25,21:35,8h 50m,0,No info,2.745580,101.709999,-33.946098,151.177002
7,Multiple Carriers,01/04/2022,Sydney,Fukuoka,Sydney-Fukuoka,21:45,08:50,13h 5m,1,No info,33.585899,130.451004,-33.946098,151.177002
8,Multiple Carriers,01/04/2022,Sydney,Chiang Mai,Sydney-Chiang Mai,16:00,14:55,26h 55m,1,No info,18.766800,98.962601,-33.946098,151.177002
9,Multiple Carriers,01/04/2022,Sydney,Da Nang,Sydney-Da Nang,16:00,15:45,27h 45m,1,No info,16.043900,108.198998,-33.946098,151.177002


In [41]:
# export to csv
df1_formatted.to_csv('../data/df1.csv')

In [42]:
# front_end code snippet 

data_processed = open('../data/df1.csv')

url = 'https://smackbang-image-w76hg6ifha-ew.a.run.app/predict'
files = {
        'file': ('data_processed.csv', data_processed, 'multipart/form-data', {
            'Expires': '0'
        })
    }
response = requests.post(url, files=files).json()

In [ ]:
pd.DataFrame(response.json())